## Setup File

Import function

In [8]:
import os
import shutil
import zipfile
import pickle
import re
import csv
import pandas as pd
import numpy as np

Create Folders if not existing

In [ ]:
data_path = 'data'
temp_path = data_path + '/temp'
backup_path = data_path + '/backup'

if not os.path.isdir(data_path):
    os.mkdir(data_path)
    os.mkdir(temp_path)
    os.mkdir(backup_path)

## Preprocessing

Preprocess Robotic arms trejectory

In [140]:
with open('orig_traj.txt', 'r') as file:
    try:
        data = file.read()
        file.close()
    except UnicodeDecodeError:
        file.close()
data = data.splitlines()
data = data[4:len(text)-1]

traj_data = []
for file in data:
    traj_data = traj_data + [[float(n) for n in file.split()]]

Preprocess Power Measurements

In [141]:
f = open('test_file.csv')

data = []
for line in f:
    data_line = line.rstrip().split('\t')
    data.append(data_line)

power_data = []
for file in data[9:]:
    file = file[0].replace(',','.')
    file = file.replace(' ','')
    power_data = power_data + [[float(n) for n in file.split(';')[:len(file.split(';'))-1]]]

Save the paramters

In [142]:
print(traj_data[3])

[0.036, -182.99879, -90.006176, 90.000475, 0.002928, -89.985901, 180.010717]


In [143]:
print(power_data)

[[0.0, -24.9915122986, -8.5890293121, 0.6828308105, -0.6437301636, -0.8153915405, 0.3683567047], [0.010000126, 24.6267318726, 8.2275867462, 0.6759166718, -0.2920627594, -1.1658668518, 0.2920627594], [0.020000252, -25.0861644745, -8.5134506226, 0.6401538849, -0.6723403931, -0.946521759, 0.3635883331], [0.030000378, 24.8124599457, 8.0564022064, 0.6592273712, -0.1239776611, -1.2302398682, 0.4601478577], [0.0400005039, -25.1944065094, -8.3086490631, 0.6401538849, -0.5507469177, -0.9417533875, 0.4661083221], [0.0500006299, 24.863243103, 7.9159736633, 0.608921051, -0.1633167267, -1.2564659119, 0.3790855408], [0.0600007559, -25.3412723541, -8.1746578217, 0.6206035614, -0.6198883057, -0.9942054749, 0.4053115845], [0.0700008819, 25.0463485718, 7.72356987, 0.6446838379, -0.5173683167, -1.3840198517, 0.524520874], [0.0800010079, -25.3705978394, -8.1005096436, 0.6439685822, -0.8642673492, -1.026391983, 0.4124641418], [0.0900011339, 25.1178741455, 7.4954032898, 0.6520748138, -0.2944469452, -1.30295